In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os, pickle
from datetime import datetime

In [3]:
# Lectura del dataset
testing_df = pd.read_csv('testing.csv')
testing_df.head()


,OPERA,SIGLADES,Fecha-I,atraso_15
0,Latin American Wings,Concepcion,2017-05-16 18:00:00,0
1,Sky Airline,Calama,2017-11-27 13:00:00,0
2,Grupo LATAM,Sao Paulo,2017-10-25 13:55:00,1
3,Sky Airline,Calama,2017-07-22 12:15:00,0
4,Sky Airline,Iquique,2017-03-18 13:35:00,0


In [9]:
def load_pickles_from_directory(directory):
    """
    Cargar todos los archivos pkl en el directorio especificado

    Args:
        directory (str): Directorio que contiene pkl files

    Returns:
        dict: Un diccionario que contiene los objetos pkl cargados, donde las claves son los nombres de archivo (sin la extensión '.pkl').
    """
    pickles = {}
    for filename in os.listdir(directory):
        if filename.endswith('.pkl'):
            path = os.path.join(directory, filename)
            with open(path, 'rb') as file:
                pickled_object = pickle.load(file)
            key = filename[:-4]  # remove the '.pkl' extension from the filename
            pickles[key] = pickled_object
    return pickles


#Funcion para crear periodo del dia
def get_periodo_dia(fecha):
    fecha_time = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S').time()
    mañana_min = datetime.strptime("05:00", '%H:%M').time()
    mañana_max = datetime.strptime("11:59", '%H:%M').time()
    tarde_min = datetime.strptime("12:00", '%H:%M').time()
    tarde_max = datetime.strptime("18:59", '%H:%M').time()
    noche_min1 = datetime.strptime("19:00", '%H:%M').time()
    noche_max1 = datetime.strptime("23:59", '%H:%M').time()
    noche_min2 = datetime.strptime("00:00", '%H:%M').time()
    noche_max2 = datetime.strptime("4:59", '%H:%M').time()
    
    if(fecha_time > mañana_min and fecha_time < mañana_max):
        return 'mañana'
    elif(fecha_time > tarde_min and fecha_time < tarde_max):
        return 'tarde'
    elif((fecha_time > noche_min1 and fecha_time < noche_max1) or
         (fecha_time > noche_min2 and fecha_time < noche_max2)):
        return 'noche'
    
def get_delay_ratios(dataset, data_pkl, tables):
    for table in tables:
        data = data_pkl[table]
        try:
            dataset = dataset.merge(data, how='left', on=data.columns[0])
        except:
            pass
    return dataset

In [10]:
#Cargar tablas
directory = '../main/utils/delay_ratio_tables'
utils_tables = load_pickles_from_directory(directory)
tables = utils_tables.keys()

In [11]:
Y = testing_df.pop('atraso_15')

In [12]:
#Columnas necesarias para predecir
cols = ['historical_delay_ratio_SIGLADES', 'historical_delay_ratio_OPERA',
       'historical_delay_ratio_MES', 'historical_delay_ratio_periodo_dia',
       'Last 1 Months_delay_ratio_OPERA', 'Last 1 Months_delay_ratio_DIA',
       'Last 3 Months_delay_ratio_DIA']

#Crear dataset para predecir
testing_df['periodo_dia'] = testing_df['Fecha-I'].apply(get_periodo_dia)
testing_df['DIA'] = pd.to_datetime(testing_df['Fecha-I']).dt.day
testing_df['MES'] = pd.to_datetime(testing_df['Fecha-I']).dt.month
testing_df = get_delay_ratios(testing_df, utils_tables, tables)
testing_df = testing_df[cols]

In [13]:
testing_df = testing_df.iloc[0]
testing_df

historical_delay_ratio_SIGLADES       0.156023
historical_delay_ratio_OPERA          0.406810
historical_delay_ratio_MES            0.157824
historical_delay_ratio_periodo_dia    0.197525
Last 1 Months_delay_ratio_OPERA       0.423729
Last 1 Months_delay_ratio_DIA         0.288770
Last 3 Months_delay_ratio_DIA         0.240066
Name: 0, dtype: float64

In [14]:
#Cargar modelo para predecir
with open('../main/model/lgbm_model_late_flight_v1.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [15]:
#Prediccion
probs = loaded_model.predict(testing_df)
y_pred = [1 if x > 0.5 else 0 for x in probs]

In [16]:
probs

array([0.27668509])

In [76]:
#Performance en testing sobre pipeline
from sklearn.metrics import confusion_matrix, classification_report
print('Reporte de clasificacion en training: \n', classification_report(Y, y_pred))

Reporte de clasificacion en training: 
               precision    recall  f1-score   support

           0       0.83      0.99      0.90     11067
           1       0.68      0.13      0.22      2574

    accuracy                           0.82     13641
   macro avg       0.76      0.56      0.56     13641
weighted avg       0.80      0.82      0.77     13641

